#### 1. HbaseS的特点是什么？
* hbase是一个分布式的，基于列簇存储的数据库. 使用hadoop的hdfs存储，使用zookeeper进行管理。
* hbase 适合存储半结构化或非结构化的数据:  
  比如数据结构字段不够确定或者杂乱无章很难按照一个概念去抽取的数据。
* hbase的存储效率比较高，为null的数据不会被存储。
* hbase的表包含rowKey、列族和列，存储数据的最小单元是`Cell`，`Cell`包含数据及其对应的写入时间戳，新写入数据时，附带写入时间戳，可以查询到之前写入的多个版本。
* hbase是`master-slave`结构，hmaster作为主节点，hregionServer作为从节点。

#### 2. Hbase中scan和get的功能以及实现的异同
* `Get`的功能是精准查找，按指定RowKey 获取唯一一条记录。
* `Scan`的功能是范围查找，按指定的条件获取一批记录。   
    * scan可以通过`setCaching`与`setBatch`方法提高速度（以空间换时间）
    * scan可以通过`setStartRow`与`setEndRow`来限定范围。范围越小，性能越高。
    * scan可以通过`setFilter`方法添加过滤器，这也是分页(性能差)、多条件查询的基础

实际上它们的实现是一样的，get操作就是一种特殊的scan（begin和end相同的scan操作）。而且hbase读数据的操作都是scan，代码级别实现的是scan，并没有特别针对get的操作。

#### 3. 请详细描述Hbase中一个Cell的结构？
cell：由{row key, column(=<family> + <label>), version}唯一确定的单元，cell中的数据是没有类型的，全部是字节码形式存储。
    
#### 4. 简述compact
HBASE中compact用途是什么，什么时候触发，分为哪两种，有什么区别，有哪些相关配置参数?   
1. 在 `hbase`中每当有`memstore`数据`flush`到磁盘之后，就形成一个`storefile`，当`storeFile`的数量达到一定程度后，就需要将`storefile`文件来进行`compaction`操作。   
2. Compact 的作用：
    * 合并文件
    * 清除过期，多余版本的数据
    * 提高读写数据的效率

3. HBase 中实现了两种 compaction 的方式：`minor` and `major`。   
   这两种 compaction 方式的区别是：   
    * `Minor` 操作只用来做部分文件的合并操作以及包括 minVersion=0，并且设置 ttl 的过期版本清理，不做任何删除数据、多版本数据的清理工作。
    * `Major` 操作是对 Region 下的 HStore 下的所有 StoreFile 执行合并操作，最终的结果是整理合并出一个文件。(删除版本号太老的数据)
    
#### 5. 描述HBase HMaster HA故障转移过程？
宕机分为 HMaster 宕机和 HRegisoner 宕机: 
* 如果是 HRegisoner 宕机， HMaster 会将其所管理的 region 重新分布到其他活动的 RegionServer 上，由于数据和日志都持久在 HDFS中，该操作不会导致数据丢失。所以数据的一致性和安全性是有保障的。
* 如果是 HMaster 宕机，会通过Zookeeper 的 Master Election 机制重新选出一个正在运行的Master 进程作为活跃节点，继续提供服务。






